In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Dark Pattern predicate labeling (single MERGED output; no intermediate split files).

- Reads:   data/processed/template.csv           (default; change with --input)
- Writes:  data/processed/merged_output.csv      (single merged file)
- Uses:    OPENAI_API_KEY from .env (python-dotenv)

Usage:
  pip install openai>=1.0.0 python-dotenv pandas
  python merged_labeler.py --model gpt-4.1-mini
  # 특정 타입만
  python merged_labeler.py --types "Urgency,Scarcity"
"""

import os
import re
import json
import time
import argparse
from typing import Dict, List, Any

import pandas as pd
from dotenv import load_dotenv

# --- Load .env so OPENAI_API_KEY is available to OpenAI() ---
load_dotenv()

# ---------------- Config ----------------
DEFAULT_INPUT = os.path.join("data", "processed", "template.csv")
DEFAULT_MODEL = "gpt-4o"
OUT_PATH     = os.path.join("data", "processed", "merged_output.csv")  # 단일 병합 파일

CANONICAL_TYPES = [
    "Urgency",
    "Misdirection",
    "Social Proof",
    "Scarcity",
    "Not Dark Pattern",
]

# Map common typos/variants to canonical
TYPE_NORMALIZE = {
    "urgency": "Urgency",
    "misdirection": "Misdirection",
    "social proof": "Social Proof",
    "social_proof": "Social Proof",
    "scarcity": "Scarcity",
    "not dark pattern": "Not Dark Pattern",
    "not_dark_pattern": "Not Dark Pattern",
    "not darkpattern": "Not Dark Pattern",
    "notdarkpattern": "Not Dark Pattern",
    "no dark pattern": "Not Dark Pattern",
    "none": "Not Dark Pattern",
}

# Allowed predicates per Type (exact strings)
PREDICATES: Dict[str, List[str]] = {
    "Urgency": [
        "Countdown Timers",
        "Limited-time Messages",
    ],
    "Misdirection": [
        "Confirmshaming",
        "Trick Questions",
        "Pressured Selling",
    ],
    "Social Proof": [
        "Activity Notifications",
        "Testimonials of Uncertain Origin",
    ],
    "Scarcity": [
        "Low-stock Messages",
        "High-demand Messages",
    ],
    "Not Dark Pattern": [
        "None"
    ]
}

# Detailed definitions (from your spec)
DEFINITIONS = {
    "Urgency": {
        "Definition": ("When a user is placed under time pressure, they are less able to critically "
                       "evaluate the information shown to them because they have less time and may "
                       "experience anxiety or stress. Providers can use this to their advantage, to push "
                       "them into completing an action that may not entirely be in the user's interest."),
        "Predicates": {
            "Countdown Timers": "Indicating to users that a deal or discount will expire using a counting-down timer",
            "Limited-time Messages": "Indicating to users that a deal or sale will expire will expire soon without specifying a deadline",
        },
    },
    "Misdirection": {
        "Predicates": {
            "Confirmshaming": "Using language and emotion (shame) to steer users away from making a certain choice",
            "Trick Questions": "Using confusing language to steer users into making certain choices",
            "Pressured Selling": "Pre-selecting more expensive variations of a product, or pressuring the user to accept the more expensive variations of a product and related products",
        },
    },
    "Social Proof": {
        "Definition": ("Social Proof is a dark pattern that leverages social cues to influence user behavior. "
                       "It creates the perception that other people are already taking an action, which pressures "
                       "users to conform. This can distort independent decision-making by making choices appear "
                       "more common or trustworthy than they actually are."),
        "Predicates": {
            "Activity Notifications": ("Shows real-time or simulated notifications about other users’ activities "
                                       "(e.g., “5 people just purchased this item”). These cues are often exaggerated "
                                       "or fabricated to pressure users into taking quick action."),
            "Testimonials of Uncertain Origin": ("Presents reviews, ratings, or endorsements that lack verifiable "
                                                 "sources or authenticity. They are designed to build false trust and "
                                                 "persuade users to follow the supposed behavior of others."),
        },
    },
    "Scarcity": {
        "Definition": ("Scarcity is a dark pattern that manipulates users by creating a false sense of limited "
                       "availability. It pressures users into making decisions quickly by suggesting that a product "
                       "or service may soon be unavailable. This tactic exploits fear of missing out (FOMO) to reduce "
                       "thoughtful decision-making."),
        "Predicates": {
            "Low-stock Messages": ("Displays warnings such as “Only 2 items left in stock” to pressure users into "
                                   "purchasing. These messages may exaggerate or fabricate stock levels to induce urgency."),
            "High-demand Messages": ("Shows claims like “This product is in high demand” or “50 people are viewing this "
                                     "right now.” Such messages create a sense of competition and urgency, often without "
                                     "reliable evidence."),
        },
    },
    "Not Dark Pattern": {
        "Predicates": {
            "None": "Text that does not represent a dark pattern for this taxonomy.",
        },
    },
}

# ------- Strong system prompt (contract-first) -------
SYSTEM_PROMPT = (
    "ROLE: Dark-Pattern Expert Annotator.\n"
    "OUTPUT CONTRACT (MANDATORY): Return ONLY a single JSON object that validates the provided JSON Schema. "
    "NO prose, NO markdown, NO explanations. "
    "If uncertain, still choose the closest predicate and lower confidence.\n"
    "QUALITY: Be concise. Rationale <= 200 chars.\n"
)

USER_TEMPLATE = """Assign the most specific predicate for the given text, constrained by the provided Type and its allowed predicates.

Type: {type_name}

Definitions:
{type_def_block}

Allowed predicates (choose exactly ONE, return the exact string):
{allowed_list}

Rules:
- Pick exactly one allowed predicate for the given Type.
- If the Type is "Not Dark Pattern", always return predicate "None".
- Be conservative: if unclear, pick the closest predicate and lower confidence.
- Base your decision ONLY on the text.

Text:
\"\"\"
{snippet}
\"\"\""""

# --------------- Helpers ---------------
def normalize_type(t: str) -> str:
    key = str(t).strip().lower()
    return TYPE_NORMALIZE.get(key, t).strip()

def build_type_block(type_name: str) -> str:
    info = DEFINITIONS.get(type_name, {})
    parts = []
    if "Definition" in info:
        parts.append(f"- Definition: {info['Definition']}")
    preds = info.get("Predicates", {})
    if preds:
        parts.append("- Predicate definitions:")
        for p, d in preds.items():
            parts.append(f"  * {p}: {d}")
    return "\n".join(parts)

def build_user_prompt(type_name: str, snippet: str) -> str:
    allowed = PREDICATES[type_name]
    allowed_list = "\n".join(f"- {p}" for p in allowed)
    block = build_type_block(type_name)
    return USER_TEMPLATE.format(
        type_name=type_name,
        type_def_block=block,
        allowed_list=allowed_list,
        snippet=snippet
    )

def ensure_parent(path: str):
    os.makedirs(os.path.dirname(path), exist_ok=True)

def make_key(text: str, t: str) -> str:
    return f"{text}||{t}"

def load_existing_keys(path: str) -> Dict[str, bool]:
    if not os.path.exists(path):
        return {}
    try:
        prev = pd.read_csv(path)
    except Exception:
        return {}
    if not {"String", "Type"}.issubset(set(prev.columns)):
        return {}
    return {make_key(str(r["String"]), str(r["Type"])): True for _, r in prev.iterrows()}

def append_rows(path: str, rows: List[dict]):
    ensure_parent(path)
    df_new = pd.DataFrame(rows)
    if os.path.exists(path):
        base = pd.read_csv(path)
        merged = pd.concat([base, df_new], ignore_index=True)
    else:
        merged = df_new
    merged.to_csv(path, index=False, encoding="utf-8")

# --------- OpenAI plumbing (Responses API + JSON Schema) ---------
def get_openai_client():
    try:
        from openai import OpenAI
    except Exception as e:
        raise RuntimeError("Install openai>=1.0.0: pip install openai") from e
    return OpenAI()

def build_json_schema(allowed_predicates: List[str]) -> dict:
    return {
        "name": "PredicateLabel",
        "schema": {
            "type": "object",
            "properties": {
                "predicate": {"type": "string", "enum": allowed_predicates},
                "confidence": {"type": "number", "minimum": 0.0, "maximum": 1.0},
                "rationale": {"type": "string", "maxLength": 200}
            },
            "required": ["predicate", "confidence", "rationale"],
            "additionalProperties": False
        },
        "strict": True
    }

def _coerce_result(data: dict, allowed_predicates: list) -> dict:
    """모델 응답을 안전하게 보정: 누락 키/형식/범위/오탈자 처리."""
    out = {}

    # predicate
    pred = str(data.get("predicate", "")).strip()
    if pred == "Pressured Seliing":
        pred = "Pressured Selling"
    if pred not in allowed_predicates:
        # 허용 셋에 없으면 가장 가까운 후보로 보정
        # Not Dark Pattern이면 None 고정
        pred = "None" if "None" in allowed_predicates else allowed_predicates[0]
    out["predicate"] = pred

    # confidence
    conf = data.get("confidence", None)
    try:
        conf = float(conf)
    except Exception:
        # 텍스트에서 confidence 유추 시도 (e.g., "confidence: 0.62")
        import re
        m = re.search(r"confidence\s*[:=]\s*([01](?:\.\d+)?)", json.dumps(data), flags=re.I)
        if m:
            conf = float(m.group(1))
        else:
            conf = 0.5  # 기본값
    # 0~1 클리핑
    if conf < 0.0: conf = 0.0
    if conf > 1.0: conf = 1.0
    out["confidence"] = conf

    # rationale
    rat = str(data.get("rationale", "")).strip()
    if not rat:
        rat = "Auto-filled rationale due to missing model field."
    out["rationale"] = rat[:300]
    return out


def call_openai(model: str, system_prompt: str, user_prompt: str, allowed_predicates: list) -> dict:
    from openai import OpenAI
    client = OpenAI()

    backoff = 1.0
    max_backoff = 30.0

    # 공통 메시지
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]

    # 함수 스키마 (tools/functions 겸용)
    json_schema = {
        "type": "object",
        "properties": {
            "predicate": {"type": "string", "enum": allowed_predicates},
            "confidence": {"type": "number", "minimum": 0.0, "maximum": 1.0},
            "rationale": {"type": "string", "maxLength": 200},
        },
        "required": ["predicate", "confidence", "rationale"],
        "additionalProperties": False,
    }
    func_def_tools = [{
        "type": "function",
        "function": {
            "name": "set_predicate",
            "description": "Label the text with an allowed predicate, confidence (0..1), and a short rationale.",
            "parameters": json_schema,
        }
    }]
    func_def_functions = [{
        "name": "set_predicate",
        "description": "Label the text with an allowed predicate, confidence (0..1), and a short rationale.",
        "parameters": json_schema,
    }]

    while True:
        try:
            # 1) 최신 방식: tools + tool_choice (일부 SDK만 지원)
            try:
                resp = client.chat.completions.create(
                    model=model,
                    messages=messages,
                    temperature=0.0,
                    tools=func_def_tools,
                    tool_choice={"type": "function", "function": {"name": "set_predicate"}},
                )
                msg = resp.choices[0].message
                if getattr(msg, "tool_calls", None):
                    args = msg.tool_calls[0].function.arguments
                    data = json.loads(args)
                    return _coerce_result(data, allowed_predicates)
                # tool_calls가 없으면 아래로 폴백
            except TypeError:
                pass  # tools 미지원 → functions로 폴백

            # 2) 구버전: functions + function_call
            try:
                resp = client.chat.completions.create(
                    model=model,
                    messages=messages,
                    temperature=0.0,
                    functions=func_def_functions,
                    function_call={"name": "set_predicate"},
                )
                msg = resp.choices[0].message
                if getattr(msg, "function_call", None):
                    args = msg.function_call.arguments
                    data = json.loads(args)
                    return _coerce_result(data, allowed_predicates)
                # function_call이 없으면 아래로 폴백
            except TypeError:
                pass  # functions 미지원 → 일반 텍스트 폴백

            # 3) 일반 텍스트 → JSON 파싱 시도 후 보정
            resp = client.chat.completions.create(
                model=model,
                messages=messages,
                temperature=0.0,
            )
            text = resp.choices[0].message.content or ""
            # 코드펜스 제거
            text = re.sub(r"^```(?:json)?\s*|\s*```$", "", text, flags=re.I | re.M).strip()
            try:
                data = json.loads(text)
            except Exception:
                # 매우 보수적으로 텍스트에서 predicate 라인만 추출 시도
                # 예: "predicate: Countdown Timers"
                import re
                m = re.search(r"predicate\s*[:=]\s*([^\n\r]+)", text, flags=re.I)
                pred_guess = m.group(1).strip() if m else ""
                data = {"predicate": pred_guess, "confidence": 0.5, "rationale": text[:200]}
            return _coerce_result(data, allowed_predicates)

        except Exception as e:
            msg = str(e).lower()
            if any(s in msg for s in ["rate", "timeout", "overloaded", "temporarily", "503", "502"]):
                import time
                time.sleep(backoff)
                backoff = min(max_backoff, backoff * 2.0)
                continue
            raise




# --------------- Core ---------------
def label_merged(df: pd.DataFrame, target_types: List[str], model: str):
    ensure_parent(OUT_PATH)
    done_keys = load_existing_keys(OUT_PATH)

    for type_name in target_types:
        subset = df[df["Type"] == type_name].copy()
        if subset.empty:
            print(f"[skip] No rows for Type={type_name}")
            continue

        allowed = list(PREDICATES[type_name])  # JSON schema enum needs a list
        rows_buffer: List[dict] = []
        total = len(subset)
        processed = 0

        for _, row in subset.iterrows():
            text = str(row["String"]).strip()
            key = make_key(text, type_name)
            if key in done_keys:  # resumable: skip already processed
                processed += 1
                continue

            prompt = build_user_prompt(type_name, text)
            result = call_openai(model, SYSTEM_PROMPT, prompt, allowed_predicates=allowed)

            pred = result.get("predicate", "")
            if pred not in allowed:
                pred = "None" if type_name == "Not Dark Pattern" else allowed[0]

            out_row = {
                "String": text,
                "Type": type_name,
                "label": int(row.get("label", 0)),
                "predicate": pred,
                "confidence": float(result.get("confidence", 0.0)),
                "rationale": str(result.get("rationale", ""))[:300],
            }
            rows_buffer.append(out_row)
            done_keys[key] = True
            processed += 1

            # Periodic flush
            if len(rows_buffer) >= 50:
                append_rows(OUT_PATH, rows_buffer)
                rows_buffer.clear()
                print(f"  ...[{type_name}] progress {processed}/{total} → {OUT_PATH}")

        if rows_buffer:
            append_rows(OUT_PATH, rows_buffer)
            rows_buffer.clear()
        print(f"[done] {type_name}: appended to {OUT_PATH}")

def main():
    ap = argparse.ArgumentParser(description="Merged predicate labeling using OpenAI Responses API (single CSV output).")
    ap.add_argument("--input", default=DEFAULT_INPUT, help="Path to CSV (default: data/processed/template.csv)")
    ap.add_argument("--model", default=DEFAULT_MODEL, help="OpenAI model (default: gpt-4.1-mini)")
    ap.add_argument("--types", default="all", help='Comma-separated list of Types or "all"')
    args = ap.parse_args()

    # Load input
    df = pd.read_csv(args.input)

    # Normalize and filter to canonical 5
    df["Type"] = df["Type"].map(lambda x: normalize_type(x))
    df = df[df["Type"].isin(CANONICAL_TYPES)].copy()

    # Pick target types
    if args.types.strip().lower() == "all":
        target_types = CANONICAL_TYPES
    else:
        raw_parts = [t.strip() for t in args.types.split(",") if t.strip()]
        target_types = []
        for p in raw_parts:
            n = normalize_type(p)
            if n not in CANONICAL_TYPES:
                raise ValueError(f"Unknown Type '{p}'. Must be one of: {CANONICAL_TYPES}")
            target_types.append(n)

    label_merged(df, target_types, args.model)
    print(f"[OK] All done. Merged output at: {OUT_PATH}")

if __name__ == "__main__":
    import sys
    # Jupyter Notebook에서 실행될 때는 불필요한 인자 제거
    if "ipykernel_launcher" in sys.argv[0]:
        sys.argv = [sys.argv[0], "--input", "../data/processed/template.csv", "--model", "gpt-4o"]
    main()



ValueError: Missing key: confidence